# Extracting Physics Using the Unscented Kalman Filter

Now that we have generated data up through phase 3, we are ready to extract best estimate physics parameters from the particle trajectories. We won't go into to much detail here other than to say that the core priciple is to minimize the intial parameters used to solve the differential equations describing the motion of the particles in the gas (including energy loss effects). This can be rather complicated. To try and make this problem tractable, we use the Unscented Kalman Filter from filterpy. Kalman filters are a whole area of research; we use the Unscented filter because it is well suited to non-linear systems.

First, all of our imports

In [ ]:
import sys
sys.path.append('..')
from pcutils.core.config import load_config
from pcutils.core.workspace import Workspace
from pcutils.core.clusterize import Cluster
from pcutils.core.estimator import Direction
from pcutils.core.particle_id import load_particle_id
from pcutils.core.target import Target
from pcutils.core.solver_kalman import apply_kalman_filter, QBRHO_2_P, Guess
from pcutils.core.kalman_args import set_kalman_args

import polars as pl
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from scipy import constants

Now, as usual, we load our configuration

In [ ]:
config = load_config('../local_config.json')
ws = Workspace(config.workspace)
nuc_map = ws.get_nuclear_map()
pid = load_particle_id(ws.get_gate_file_path(config.solver.particle_id_filename), nuc_map)
target = Target(config.solver.gas_data_path, nuc_map)

Notice here we've loaded not only our workspace but also our paritcle ID data and a target. These are important! Now that we're doing physics we need to pick slices of data that correspond to specific particles so that we can select the appropriate charge and mass. The target allows us to calculate energy loss.

Now we load up our cluster data as well as our estimates from phase 3.

In [ ]:
run_number = 4
cluster_file = h5.File(ws.get_cluster_file_path(run_number))
estimate_df = pl.scan_parquet(ws.get_estimate_file_path_parquet(run_number))
estimate_gated = estimate_df.filter(pl.struct(['dEdx', 'brho']).map(pid.cut.is_cols_inside)).collect().to_dict()
cluster_group = cluster_file['cluster']
nrows = len(estimate_gated['event'])
row = np.random.randint(0, nrows)
#row = 587
print(f'row: {row}')
event = estimate_gated['event'][row]
cluster_index = estimate_gated['cluster_index'][row]
print(f'event: {event}')
print(f'cluster index: {cluster_index}')
event_group = cluster_group[f'event_{event}']
local_cluster = event_group[f'cluster_{cluster_index}']
print(f'Direction: {estimate_gated["direction"][row]}')
cluster = Cluster(event, local_cluster.attrs['label'], local_cluster['cloud'][:].copy())
cluster.z_bin_width = local_cluster.attrs['z_bin_width']
cluster.z_bin_low_edge = local_cluster.attrs['z_bin_low_edge']
cluster.z_bin_hi_edge = local_cluster.attrs['z_bin_hi_edge']
cluster.n_z_bins = local_cluster.attrs['n_z_bins']
# Rescale to m
cluster.data[:, :3] *= 0.001

As per usual we've selected a random cluster to look at. Now we create a Guess from our estimated parameters. We also handle flipping the data if the direction is backwards (>90 degrees lab).

In [ ]:
guess = Guess()
guess.brho = estimate_gated['brho'][row]
guess.polar = estimate_gated['polar'][row]
guess.azimuthal = estimate_gated['azimuthal'][row]
#again rescale to meters
guess.vertex_x = estimate_gated['vertex_x'][row] * 0.001
guess.vertex_y = estimate_gated['vertex_y'][row] * 0.001
guess.vertex_z = estimate_gated['vertex_z'][row] * 0.001
guess.direction = Direction(estimate_gated['direction'][row])
if guess.direction is Direction.BACKWARD:
    np.flip(cluster.data, axis=0)

Now we pass the extra filter arguments to a special class.

In [ ]:
Bfield = -1.0 * config.detector.magnetic_field
Efield = -1.0 * config.detector.electric_field
set_kalman_args(target, pid.nucleus, Bfield, Efield)

Annnnnd, then we run the Unscented Kalman Filter!

In [ ]:
trajectory, covariances = apply_kalman_filter(cluster.data[:,:3], cluster.z_bin_width * 0.001, guess)

The filter returns to us the filtered, smoothed trajectory and the covariances. In principle, this is the best fit of our equations to the data. Now we can plot the trajectory against the data, as shown below

In [ ]:
plt.scatter(cluster.data[:, 0], cluster.data[:, 1].copy(), s=2, label='data')
plt.scatter(trajectory[:, 0], trajectory[:, 1], s=2, label='filter')
plt.scatter(trajectory[0, 0], trajectory[0, 1], s=2, label='filter 0')


In [ ]:
plt.scatter(cluster.data[:, 2], cluster.data[:, 1], s=2, label='data')
plt.scatter(trajectory[:, 2], trajectory[:, 1], s=2, label='filter')
plt.scatter(trajectory[0, 2], trajectory[0, 1], s=2, label='filter 0')


In [ ]:
plt.scatter(cluster.data[:, 2], cluster.data[:, 0], s=2, label='data')
plt.scatter(trajectory[:, 2], trajectory[:, 0], s=2, label='filter')
plt.scatter(trajectory[0, 2], trajectory[0, 0], s=2, label='filter 0')

We can also compare the our original guess of the initial position to the trajetory estimated initial position (as well as energy and angles)

In [ ]:
print(f'Initial guess: {guess}')
buest_guess = Guess()
buest_guess.polar = np.arctan2(np.linalg.norm(trajectory[0, 3:5]), trajectory[0, 5])
buest_guess.azimuthal = np.arctan2(trajectory[0, 4], trajectory[0, 3])
if buest_guess.azimuthal < 0:
    buest_guess.azimuthal += np.pi * 2.0
buest_guess.vertex_x = trajectory[0, 0]
buest_guess.vertex_y = trajectory[0, 1]
buest_guess.vertex_z = trajectory[0, 2]
best_momentum = pid.nucleus.mass * (np.linalg.norm(trajectory[0, 3:]) / constants.speed_of_light)
buest_guess.brho = best_momentum / (QBRHO_2_P * pid.nucleus.Z)
print(f'Best guess: {buest_guess}')

Now we have a best estimate for the physics from this particular cluster, using the Kalman filter!